# Data Preparation

In [1]:
#importing libraries:

import numpy as np
from pandas import Series,DataFrame
import pandas as pd
import datetime as dt
import janitor
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


Importing All Flat files for analysis:

In [2]:
#Contracts:
contracts =  pd.read_excel('Contracts June Snapshot -Active and ND.xlsx')
contracts =  pd.DataFrame(contracts)

# Creating a coulumn for contract_line_number:
contracts['contract_line_number'] = contracts.ContractNumber.astype(str) + "-" + contracts.ContractItemNumber.astype(str)

# Selecting only VBR and Support Services records and restoring contracts to those:
contracts_filtered = contracts[contracts['Portfolio'].isin(["VBR", "Support Services"])]
contracts = pd.DataFrame(contracts_filtered)

contracts.get_dtype_counts()

datetime64[ns]     4
float64           12
int64              7
object            44
dtype: int64

In [3]:
contracts.Portfolio.unique()

array(['Support Services', 'VBR'], dtype=object)

In [4]:
#ITSM:
itsm = pd.read_excel('itsm.xlsx').clean_names()

#Manfacturer_mapping:
manu_mapping =  pd.read_excel('Manufacturer_Std_Mapping V1_new.xlsx').clean_names()


In [5]:
#PO:
po = pd.read_excel("PO_1.xlsx")
po['SalesDocumentItemNumber'] = po['SalesDocumentItemNumber'].astype(np.int64)

# Creating a coulumn for sales_document_line_number:
po['sales_document_line_number'] = po.SalesDocumentNumber.astype(str) +'-'+ po.SalesDocumentItemNumber.astype(str)

po.get_dtype_counts()

float64     3
int64       3
object     10
dtype: int64

# Calculating missing NetValueUSD for non-zero ACVBUDGETUSD:

In [6]:
mask1 = contracts["ACVUSDBudget"] != 0
mask2 = contracts["NetValueUSD"] == 0

contracts_filter = contracts[mask1 & mask2]

contracts_filter["date_diff_days"] = pd.to_datetime(contracts_filter["End Date"]) - pd.to_datetime(contracts_filter["Start Date"])
contracts_filter["ACV_per_day"] = contracts["ACVUSDBudget"]/365
contracts_filter["NetValueUSD"] = contracts_filter["ACV_per_day"] * (contracts_filter["date_diff_days"]).dt.days
contracts_filter = contracts_filter[['contract_line_number','NetValueUSD']]

#merging back to contracts:
contracts_merged_final = contracts.merge(contracts_filter, how="left", on = "contract_line_number")
contracts_merged_final["NetValueUSD_y"].fillna(contracts_merged_final["NetValueUSD_x"], inplace= True)

#Taking only non-zero (ACVBudgetUSD and NetValueUSD_y) observations:
mask1 = contracts_merged_final["ACVUSDBudget"] != 0
mask2 = contracts_merged_final["NetValueUSD_y"] != 0

contracts_merged_final = contracts_merged_final[mask1 & mask2]

#Removing NetValueUSD_x as we have adjusted net value for non zero ACVBudgetUSD in NetValueUSD_y:
contracts_merged_final.drop(['NetValueUSD_x'], axis=1,inplace = True) 

#Renaming NetValueUSD_y to NetValueUSD_x:
contracts_merged_final.rename(columns = {'NetValueUSD_y':'NetValueUSD_x'}, inplace = True)

#Restoring everything to contracts.
contracts = pd.DataFrame(contracts_merged_final) 

C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

# Imputing DD-AM,AP,EU, GLOBAL, MEA:

In [7]:
po["Manufacturer"]= po["Manufacturer"].str.replace("DD-GLOBAL", "DD", case = False)
po["Manufacturer"]= po["Manufacturer"].str.replace("DD-AM", "DD", case = False)
po["Manufacturer"]= po["Manufacturer"].str.replace("DD-AP", "DD", case = False)
po["Manufacturer"]= po["Manufacturer"].str.replace("DD-EU", "DD", case = False)
po["Manufacturer"]= po["Manufacturer"].str.replace("DD-MEA", "DD", case = False)
contracts["ManufacturerCode"]= contracts["ManufacturerCode"].str.replace("DD-GLOBAL", "DD", case = False)
contracts["ManufacturerCode"]= contracts["ManufacturerCode"].str.replace("DD-AM", "DD", case = False)
contracts["ManufacturerCode"]= contracts["ManufacturerCode"].str.replace("DD-AP", "DD", case = False)
contracts["ManufacturerCode"]= contracts["ManufacturerCode"].str.replace("DD-EU", "DD", case = False)
contracts["ManufacturerCode"]= contracts["ManufacturerCode"].str.replace("DD-MEA", "DD", case = False)

In [8]:
contracts["NetValueUSD_x"].sum()

2837275407.590391

# Mapping standard manufactureres names to ITSM from Mapping File:

In [9]:
#renaming itsm_manufacturer to manufacturer in manu_mapping
manu_mapping.rename(columns = {'itsm_manufacturer':'manufacturer'}, inplace = True)

#getting manufacturers standard names in ITSM:
itsm_mapping_merged = itsm.merge(manu_mapping, how="left", on = "manufacturer")

itsm_mapping_merged.standard_mapping.isna().sum()
# 4762 not mapped
pd.notna(itsm_mapping_merged["standard_mapping"]).sum()
# 114311 mapped

#Replacing NAs in Standard Mapping with manufacturer as given in manu_mapping file.
itsm_mapping_merged["standard_mapping"].fillna(itsm_mapping_merged["manufacturer"], inplace= True)

# Resoring to ITSM:
itsm = pd.DataFrame(itsm_mapping_merged).clean_names()


# Merging Contracts, PO and ITSM:

In [10]:
#contract_PO:
contracts_merged_po = contracts.merge(po, how="left", left_on= "contract_line_number", right_on="sales_document_line_number")
contracts_merged_po["merged_po"]= np.where(contracts_merged_po['SalesDocumentNumber'].isna() , 0, 1)

#contracts_po_itsm:
contract_po_itsm_merged = contracts_merged_po.merge(itsm[["contract_line_number","standard_mapping"]], how= "left", on="contract_line_number")
contract_po_itsm_merged["merged_itsm"]= np.where(contract_po_itsm_merged['standard_mapping'].isna() , 0, 1)

#missing itsm manufacturers in contracts_po_itsm:
pd.isna(contract_po_itsm_merged["standard_mapping"]).sum()
# 173812

#here on taking contracts as a single source of all data.
contracts = pd.DataFrame(contract_po_itsm_merged).clean_names()

# Vendor Standardisation:

In [11]:
contracts_manufac_filter = contracts[["contract_line_number","manufacturer","manufacturercode","vendor","standard_mapping"]]
vendor_mapper =pd.DataFrame(manu_mapping)

vendor_mapper.rename(columns = {'manufacturer':'vendor'}, inplace = True)
vendor_mapper.rename(columns = {'standard_mapping':'standard_mapping_vendor'}, inplace = True)
contracts_manufac_filter_1 = contracts_manufac_filter[["contract_line_number","vendor"]].merge(vendor_mapper, how="left", on = "vendor")

#Replacing NA in standardised_vendor with existing vendor missing in vendor_mapper:
contracts_manufac_filter_1["standard_mapping_vendor"].fillna(contracts_manufac_filter_1["vendor"], inplace= True)

contracts_manufac_filter_1= pd.DataFrame(contracts_manufac_filter_1.drop(contracts_manufac_filter_1.index[390786:390818]))

#Arranging Contract_line_number and Vendor for cbind:
contracts = contracts.sort_values(by=["contract_line_number","vendor"])
contracts_manufac_filter_1 = contracts_manufac_filter_1.sort_values(by=["contract_line_number","vendor"])

#cbinding standardised vendor to contracts:
contracts = contracts.drop(columns=["vendor"])
contracts_manufac_filter_1 = contracts_manufac_filter_1.drop(columns=["vendor"])
contracts = pd.concat([contracts, contracts_manufac_filter_1], axis=1)

#removing extra "Contract_Line_Number col. "
contracts=contracts.drop(columns=contracts.columns[85])
contracts['contract_line_number'] = contracts.contractnumber.astype(str) + "-" + contracts.contractitemnumber.astype(str)

# renaming standard_mapping_vendor to vendor
contracts.rename(columns = {'standard_mapping_vendor':'vendor'}, inplace = True)

# Manufacturer Mapping:

In [13]:
contracts["manufacturer_po"] = contracts["manufacturer"]
contracts = contracts.drop(columns=["manufacturer"])

# Re-assigning manufacturer: 
contracts["manufacturer"] = np.where(contracts["portfolio"]=="VBR", contracts["manufacturercode"], contracts["manufacturer_po"])

mask_1 = contracts["portfolio"]=="VBR"
mask_2 = contracts["manufacturer"].isna()
contracts["manufacturer"] = np.where(mask_1 & mask_2, contracts["vendor"], contracts["manufacturer"])

mask_3 = contracts["portfolio"]=="Support Services"
mask_4 = contracts["merged_po"] == 0
contracts["manufacturer"] = np.where(mask_3 & mask_4, contracts["standard_mapping"], contracts["manufacturer"])

#IF ALSO DD then take from vendor:
mask_5 = contracts["merged_po"] == 1
mask_6 = contracts["manufacturer"].isna()
mask_8 = contracts["manufacturer"] == "DD"
contracts["manufacturer"] = np.where(mask_3 & mask_5 & (mask_6 | mask_8), contracts["vendor"], contracts["manufacturer"])

mask_7 = contracts["manufacturer"].isna() 
mask_9 = contracts["manufacturer"] == "DD"

contracts["manufacturer"] = np.where(mask_3 & mask_5 & mask_7 | mask_9, "DD-UPTIME", contracts["manufacturer"])
contracts["manufacturer"].fillna("DD-UPTIME", inplace= True)

#merged_itsm:
contracts["merged_itsm"]= np.where(contracts["manufacturer"] == contracts["standard_mapping"],1,0)

#Removing manufacturer_po:
contracts=contracts.drop(columns=["manufacturer_po"])

# Creating TCV split up:

In [14]:
# client_country & client_country_manufacturer combinations:

#client_country:
contracts["client_country"] = contracts.adjusted_client_domain_description.astype(str) + "-" + contracts.country.astype(str)

#client_country_manufacturer:
contracts["client_country_manufacturer"] = contracts.client_country.astype(str) + "-" + contracts.manufacturer.astype(str)


In [15]:
# Separate DF for vbr and ss :
mask_1 = contracts["portfolio"]=="VBR"
mask_2 = contracts["portfolio"]=="Support Services"

df_vbr = contracts[mask_1]
df_ss  = contracts[mask_2] 

In [16]:
#Creating TCV split up: only for "Support Services".
total_spend_value = df_ss.groupby("contract_line_number").agg({"spend_value_usd" : "sum","client_country_manufacturer":"nunique"}).reset_index()
total_spend_value.rename(columns = {'spend_value_usd':'total_spend_value','client_country_manufacturer':'count'}, inplace = True)
total_spend_value = pd.DataFrame(total_spend_value)

df_ss = df_ss.merge(total_spend_value, how ="left", on="contract_line_number")
df_ss["net_value_usd_x_1"] = df_ss["netvalueusd_x"]/df_ss["count"]

#conditions = [(df_ss["spend_value_usd"]==0) | (df_ss["total_spend_value"]==0)]
#choices = [np.nan]
#df_ss["proportion"] = np.select(conditions, choices, default= (df_ss["spend_value_usd"])/(df_ss["total_spend_value"]))

df_ss["proportion"] = df_ss["spend_value_usd"]/np.where((df_ss["spend_value_usd"]==0)|(df_ss["total_spend_value"]==0),np.nan,df_ss["total_spend_value"])
pos_na = df_ss.index[df_ss["proportion"].apply(np.isnan)]
df_na = df_ss.iloc[pos_na]
df_na["cln"] = df_na["contract_line_number"]
df_na1 = df_na.groupby("contract_line_number").agg({"cln" :"count"}).reset_index()
df_na1.rename(columns = {'cln':'count'}, inplace = True)
df_na1 = pd.DataFrame(df_na1.assign(proportion_new = lambda x: 1/df_na1["count"]))
df_na2 = df_na.merge(df_na1[["contract_line_number","proportion_new"]], how="inner", on="contract_line_number" )
df_ss["proportion"][pos_na] = df_na2["proportion_new"]



C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
#Creating TCV split up: only for "VBR".
df_vbr["cln"] = df_vbr["contract_line_number"]
vbr_cli_count = df_vbr.groupby("contract_line_number").agg({"cln" :"count"}).reset_index()
vbr_cli_count.rename(columns = {'cln':'count'}, inplace = True)
vbr_cli_count = pd.DataFrame(vbr_cli_count.assign(proportion = lambda x: 1/vbr_cli_count["count"]))
vbr_cli_count = pd.DataFrame(vbr_cli_count)
df_vbr = df_vbr.merge(vbr_cli_count, how="left", on="contract_line_number")
df_vbr["net_value_usd_x_1"] = df_vbr["netvalueusd_x"]


C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
# Removing Excess cloumns in df_ss and df_vbr before appending both to one as contracts.
df_vbr=df_vbr.drop(columns=["count"])
df_vbr=df_vbr.drop(columns=["cln"])
df_ss=df_ss.drop(columns=["total_spend_value"])
df_ss=df_ss.drop(columns=["count"])

In [19]:
# Appending df_ss and df_vbr:
contracts_1 = df_ss.append(df_vbr, ignore_index=True)

C:\Users\gramener\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [20]:
# Net_Value_USD SPLIT :
contracts_1["net_value_usd_x_calc"] = contracts_1["net_value_usd_x_1"] * contracts_1["proportion"]

In [21]:
# Validation Check merged_Netvalue:
merged_netvalue = contracts_1["net_value_usd_x_calc"].sum()

vbr_netvalue = contracts_1[contracts_1['portfolio'].isin(["VBR"])]
vbr_netvalue = vbr_netvalue["net_value_usd_x_calc"].sum()

ss_netvalue = contracts_1[contracts_1['portfolio'].isin(["Support Services"])]
ss_netvalue = ss_netvalue["net_value_usd_x_calc"].sum()

In [22]:
# Total-TCV:
merged_netvalue

2772281370.1523523

In [23]:
# vbr-TCV:
vbr_netvalue

1201418632.4175606

In [24]:
# ss-TCV:
ss_netvalue

1570862737.734791

# DERIVED - METRICS


In [24]:
#client_manufacturer:
contracts_1["client_manufacturer"] = contracts_1.adjusted_client_domain_description.astype(str) + "-" + contracts_1.manufacturer.astype(str)

In [25]:
#client_country_uptime:
client_country_uptime = contracts_1[contracts_1['portfolio'].isin(["Support Services"])]
client_country_uptime["cc"] = client_country_uptime["client_country"]

client_country_uptime = client_country_uptime.groupby("client_country").agg({"cc":"count"}).reset_index()
client_country_uptime.rename(columns = {'cc':'count'}, inplace = True)
client_country_uptime = pd.DataFrame(client_country_uptime)

client_country_uptime["client_country_uptime"] = 1
client_country_uptime = client_country_uptime[["client_country", "client_country_uptime"]]

contracts_1 = contracts_1.merge(client_country_uptime, how="left", on= "client_country")
contracts_1["client_country_uptime"] = np.where(contracts_1["client_country_uptime"].isna(),0,1)

contracts_1.client_country_uptime.sum()
#343714

C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


343714

In [26]:
#client_uptime:
client_uptime = contracts_1[contracts_1['portfolio'].isin(["Support Services"])]
client_uptime["c"] = client_uptime["adjusted_client_domain_description"]

client_uptime = client_uptime.groupby("adjusted_client_domain_description").agg({"c":"count"}).reset_index()
client_uptime.rename(columns = {'c':'count'}, inplace = True)
client_uptime = pd.DataFrame(client_uptime)

client_uptime["client_uptime"] = 1
client_uptime = client_uptime[["adjusted_client_domain_description", "client_uptime"]]

contracts_1 = contracts_1.merge(client_uptime, how="left", on= "adjusted_client_domain_description")
contracts_1["client_uptime"] = np.where(contracts_1["client_uptime"].isna(),0,1)

contracts_1.client_uptime.sum()
#361458

C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


361458

In [47]:
#contracts_copy = pd.DataFrame(contracts_1)

In [27]:
#multiple_overall:
multiple_overall = contracts_1.groupby("client_country").agg({"manufacturer":"nunique"}).reset_index()
multiple_overall.rename(columns = {'manufacturer':'count'}, inplace = True)
multiple_overall = multiple_overall[multiple_overall["count"]>1]
multiple_overall = pd.DataFrame(multiple_overall)

multiple_overall["multiple_overall"] = 1

multiple_overall = multiple_overall[["client_country", "multiple_overall"]]

contracts_1 = contracts_1.merge(multiple_overall, how="left", on= "client_country")
contracts_1["multiple_overall"] = np.where(contracts_1["multiple_overall"].isna(),0,1)
contracts_1.multiple_overall.sum()
#351005

351005

In [28]:
#client_country_manufacturer_both_1_uptime_2_vbr_3 
client_country_manufacturer_both_1_uptime_2_vbr_3 = pd.DataFrame(contracts_1)

client_country_manufacturer_both_1_uptime_2_vbr_3 = client_country_manufacturer_both_1_uptime_2_vbr_3.groupby(["client_country_manufacturer","portfolio"]).agg({"net_value_usd_x_calc":"sum"}).reset_index()
client_country_manufacturer_both_1_uptime_2_vbr_3.rename(columns = {'net_value_usd_x_calc':'tcv'}, inplace = True)
client_country_manufacturer_both_1_uptime_2_vbr_3 = pd.DataFrame(client_country_manufacturer_both_1_uptime_2_vbr_3)

#Melt net_value_c_calc into VBR and SS:
client_country_manufacturer_both_1_uptime_2_vbr_3=client_country_manufacturer_both_1_uptime_2_vbr_3.pivot_table(index=['client_country_manufacturer'],columns='portfolio',values='tcv',fill_value=0)
client_country_manufacturer_both_1_uptime_2_vbr_3["total"] = client_country_manufacturer_both_1_uptime_2_vbr_3["Support Services"] + client_country_manufacturer_both_1_uptime_2_vbr_3["VBR"]

# Nested condition for assigning 1,2,3,4:
conditions = [(client_country_manufacturer_both_1_uptime_2_vbr_3["Support Services"] != 0) & (client_country_manufacturer_both_1_uptime_2_vbr_3["VBR"]!=0),
             (client_country_manufacturer_both_1_uptime_2_vbr_3["Support Services"] != 0) & (client_country_manufacturer_both_1_uptime_2_vbr_3["VBR"]==0),
             (client_country_manufacturer_both_1_uptime_2_vbr_3["Support Services"] == 0) & (client_country_manufacturer_both_1_uptime_2_vbr_3["VBR"]!=0)]
choices = [1,2,3]
client_country_manufacturer_both_1_uptime_2_vbr_3["client_country_manufacturer_both_1_uptime_2_vbr_3"] = np.select(conditions, choices, default= 4)

#Reseting Indexcolumn:
client_country_manufacturer_both_1_uptime_2_vbr_3 = client_country_manufacturer_both_1_uptime_2_vbr_3.reset_index()
client_country_manufacturer_both_1_uptime_2_vbr_3 = client_country_manufacturer_both_1_uptime_2_vbr_3[["client_country_manufacturer","client_country_manufacturer_both_1_uptime_2_vbr_3"]]

#merge back to contracts:
client_country_manufacturer_both_1_uptime_2_vbr_3 = pd.DataFrame(client_country_manufacturer_both_1_uptime_2_vbr_3)
contracts_1 = contracts_1.merge(client_country_manufacturer_both_1_uptime_2_vbr_3, how="left", on= "client_country_manufacturer")

contracts_1["client_country_manufacturer_both_1_uptime_2_vbr_3"] = np.where(contracts_1["client_country_manufacturer_both_1_uptime_2_vbr_3"].isna(),4,contracts_1["client_country_manufacturer_both_1_uptime_2_vbr_3"])



In [29]:
# renaming:
contracts_1.rename(columns = {'manufacturerpartnumber_x':'manufacturer_part_number'}, inplace = True)
#removing standard_mapping:
contracts_1 = contracts_1.drop(columns=["standard_mapping"])


In [30]:
# Success_play 3 new logic:

sp3_data = contracts_1.groupby(["client_manufacturer","portfolio"]).agg({"net_value_usd_x_calc":"sum"}).reset_index()
sp3_data.rename(columns = {'net_value_usd_x_calc':'tcv'}, inplace = True)
sp3_data = pd.DataFrame(sp3_data)

#melt net valu into VBR and Support Services.
sp3_data=sp3_data.pivot_table(index=['client_manufacturer'],columns='portfolio',values='tcv',fill_value=0)
sp3_data = sp3_data.reset_index()
sp3_data["grand_total"] = sp3_data["Support Services"] + sp3_data["VBR"]

#grouping by client-manufacturer and getting net value for each client manufacturer group:
sp3_data = sp3_data.groupby("client_manufacturer").agg({"VBR":"sum", "Support Services":"sum"}).reset_index()
sp3_data.rename(columns = {'VBR':'total_vbr','Support Services':'total_ss'}, inplace = True)

# flagging client_manufacturer combination having both vbr and ss:

distinct_client_country = sp3_data[(sp3_data["total_vbr"]!= 0) & (sp3_data["total_ss"]!= 0)]
distinct_client_country["flag"] = 1
distinct_client_country = distinct_client_country[["client_manufacturer","flag"]]

#merging distinct_client_country with contracts_1 to get flag.
contracts_1 = contracts_1.merge(distinct_client_country, how="left", on="client_manufacturer")
contracts_1["flag"] = np.where(contracts_1["flag"].isna(),0,1)

s3_ss = contracts_1[contracts_1["portfolio"] == "Support Services"]
s3_ss = pd.unique(s3_ss["client_country_manufacturer"])
s3_ss = pd.DataFrame(s3_ss)

s3_vbr = contracts_1[contracts_1["portfolio"] == "VBR"]
s3_vbr = pd.unique(s3_vbr["client_country_manufacturer"])
s3_vbr = pd.DataFrame(s3_vbr)

s3_ss["f"] = "ss"
s3_vbr["f"] = "vbr"

s3_ss.rename(columns = {0:'client_country_manufacturer'}, inplace = True)
s3_vbr.rename(columns = {0:'client_country_manufacturer'}, inplace = True)

vbr_left = s3_vbr.merge(s3_ss, how= "left", on="client_country_manufacturer")
vbr_left["check"] = np.where(vbr_left["f_y"].isna(),1,0)
vbr_left = vbr_left[["client_country_manufacturer", "check"]]

contracts_1 = contracts_1.merge(vbr_left, how = "left", on="client_country_manufacturer")
contracts_1["check"] = np.where(contracts_1["check"].isna(),0,1)
contracts_1["check_final"] = np.where((contracts_1["flag"]==1) & (contracts_1["check"]==1) ,1,0)



C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
# Calculating acvusd_budget_calc:
contracts_1["acvusd_budget_calc"] = contracts_1["acvusdbudget"] * contracts_1["proportion"]

contracts_1.acvusd_budget_calc.sum()
# 1750717484.739311

In [35]:
# CALCULATING SUCCESS_PLAYS:

contracts_1["success_play_1"] = np.where((contracts_1["client_country_manufacturer_both_1_uptime_2_vbr_3"] ==1),1,0)

contracts_1["success_play_2"] = np.where(((contracts_1["multiple_overall"] ==1) & (contracts_1["success_play_1"] ==0) & (contracts_1["client_country_uptime"] ==1) & (contracts_1["client_country_manufacturer_both_1_uptime_2_vbr_3"] ==3)) ,1,0)

contracts_1["success_play_3"] = np.where(((contracts_1["client_country_manufacturer_both_1_uptime_2_vbr_3"] ==3) & (contracts_1["check_final"]==1)),1,0)

contracts_1["success_play_4"] = np.where((contracts_1["client_uptime"]==1),1,0)


In [39]:
#Validation Check merged_Netvalue:

merged_netvalue = contracts_1["net_value_usd_x_calc"].sum()

vbr_netvalue = contracts_1[contracts_1['portfolio'].isin(["VBR"])]
vbr_netvalue = vbr_netvalue["net_value_usd_x_calc"].sum()

ss_netvalue = contracts_1[contracts_1['portfolio'].isin(["Support Services"])]
ss_netvalue = ss_netvalue["net_value_usd_x_calc"].sum()


# Validating Success_Play Netval:

In [43]:
# SS_1
SS_1_netval = contracts_1[contracts_1["success_play_1"]==1]
SS_1_netval = SS_1_netval["net_value_usd_x_calc"].sum()
SS_1_netval

566277990.1544968

In [44]:
# SS_2
SS_2_netval = contracts_1[contracts_1["success_play_2"]==1]
SS_2_netval = SS_2_netval["net_value_usd_x_calc"].sum()
SS_2_netval

448214576.465385

In [45]:
# SS_3
SS_3_netval = contracts_1[contracts_1["success_play_3"]==1]
SS_3_netval = SS_3_netval["net_value_usd_x_calc"].sum()
SS_3_netval

291108942.02345276

In [46]:
# SS_4
SS_4_netval = contracts_1[contracts_1["success_play_4"]==1]
SS_4_netval = SS_4_netval["net_value_usd_x_calc"].sum()
SS_4_netval

2538888600.589833

In [48]:
# Creating TPIC_FLAG:client_country:

Tpic = contracts_1.groupby("client_country").agg({"manufacturer":"nunique"}).reset_index()
Tpic.rename(columns = {'manufacturer':'count'}, inplace = True)
Tpic = Tpic[Tpic["count"]>1]
Tpic["tpic"] = 1
Tpic = Tpic[["client_country","tpic"]]
Tpic = pd.DataFrame(Tpic)

contracts_1= contracts_1.merge(Tpic, how="left" , on="client_country")
contracts_1["tpic"] = np.where((contracts_1["tpic"].isna()),0,1)

In [61]:
#Output:
df1 = pd.DataFrame(contracts_1)
df1.to_excel("CLI_view_python_op.xlsx")
